###### 2020-11-16 월요일
###### 2020-11-17 화요일

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import glob
import re

from tqdm import tqdm_notebook

## 1. 댓글데이터병합 + 400자 초과댓글 제거

#### (1) 댓글데이터병합

In [3]:
file_name = glob.glob('./01_raw_data/*.csv')

In [4]:
repl_data = pd.read_csv(file_name[0])
repl_data['출처'] = file_name[0]

In [5]:
for path in file_name[1:]:
    temp_data = pd.read_csv(path)
    temp_data['출처'] = path
    repl_data = pd.concat([repl_data, temp_data])
    print(path)

./data\kbs댓글.csv
./data\mbc댓글.csv
./data\subusunews.csv
./data\ytnnews.csv
./data\고영욱.csv
./data\김계란.csv
./data\문재인.csv
./data\박성광.csv
./data\배다해.csv
./data\스윙스.csv
./data\에브리타임_댓글_1113.csv
./data\여행에 미치다.csv
./data\위키블루.csv
./data\유머 짤.csv
./data\이상민.csv
./data\이슈 콕.csv
./data\이슈이슈.csv
./data\인사이트.csv
./data\인사이트_biz.csv
./data\인사이트_ent.csv
./data\일베_일간베스트게시물_댓글_최근12페이지.csv
./data\장성규.csv
./data\청와대댓글.csv
./data\최자.csv
./data\함소원.csv
./data\홍석천.csv


## 1. 라벨링 되지않은 원본데이터 저장

#### (1) 전체데이터 저장

In [ ]:
repl_data.to_csv('./merge_data/merge_data.csv')

#### (2) 400자 초과 댓글제거

In [7]:
repl_data['댓글길이'] = repl_data['댓글'].apply(lambda x : len(str(x)))

In [8]:
repl_data = repl_data[repl_data['댓글길이'] <= 400]

## 2. 불용어(이모티콘)처리

#### (1) 중복된 댓글 제거

In [9]:
raw_data = repl_data.drop_duplicates(subset='댓글', keep='first', inplace=False)

In [10]:
# 정규표현식 전과 후를 비교하기 위해 세로운 데이터프레임 생성
preprocessing_df = raw_data.copy()

#### (2) `@[아이디]` 제거

In [11]:
preprocessing_df['댓글'] = preprocessing_df['댓글'].str.replace('@[a-zA-Z가-힣\w\.\_\d]+', '')

#### (3) `한글/영어/숫자`를 제외한 문자제거

 - 이모티콘이나 특수문자를 제거한다

In [12]:
preprocessing_df['댓글'] = preprocessing_df['댓글'].str.replace('[^a-zA-Z가-힣0-9ㄱ-ㅎㅏ-ㅣ\s]', '')

 - 빈공백 글자 `' '`로 통일
 - 이제 빈댓글을 제거할때 `''`와 `' '`로 제거한다

In [13]:
preprocessing_df['댓글'] = preprocessing_df['댓글'].str.replace('\s[\s*]', '')

#### (4) 양옆에 빈 공백 제거

In [14]:
preprocessing_df['댓글'] = preprocessing_df['댓글'].str.rstrip()

In [15]:
preprocessing_df['댓글'] = preprocessing_df['댓글'].str.lstrip()

 - 영어만 있는 행을 제거하기 전에 인덱스를 정렬하자

#### (5) 빈댓글 제거

In [16]:
preprocessing_df = preprocessing_df[preprocessing_df['댓글'] != '']

In [17]:
preprocessing_df = preprocessing_df[preprocessing_df['댓글'] != ' ']

In [18]:
preprocessing_df.shape

(108847, 4)

In [19]:
preprocessing_df[preprocessing_df['댓글'].str.isspace().isna()]

,Unnamed: 0,댓글,출처,댓글길이
1386,1386,NaN,./data\스윙스.csv,3


#### (6) 결측값제거

 - 스윙스댓글 1386번째 댓글이 NaN값이 뜬다 제거하자

In [20]:
preprocessing_df = preprocessing_df[~preprocessing_df['댓글'].isna()]

#### (7) 영어만 있는 행 제거

In [21]:
preprocessing_df = preprocessing_df.reset_index()
preprocessing_df = preprocessing_df.drop(['index', 'Unnamed: 0', '댓글길이'], axis=1, inplace=False)

In [22]:
preprocessing_df['댓글'].str.lower()

0         심각허다 법안 검토도 안 하는 의원들 내 세금으로 호의호식 한다 생각하니까 열받네 ...
1         차도에서 25km제한 둠 어쩌자는건지 자전거 오토바이도 그거보다 빠르다 인도 차도 ...
2                                       국회의원직 다 박탈했으면 좋겠네ㅠㅠ
3                                        표 얻을 생각 말고 법을 만들어라
4                              초딩 애들이 몰고다니는 형형색색 씽씽이 생각했나보네
                                ...                        
108841                                              오빠 어디예요
108842                                               외국 갔어요
108843                                 이건 홍마담에서 구입 못하나요 ㅎㅎㅎ
108844                                           느무 잘먹는 예 9
108845                               맛있는건 눈으로만 봐도 알죠맛있게 드세요
Name: 댓글, Length: 108846, dtype: object

In [23]:
preprocessing_df = preprocessing_df[~preprocessing_df['댓글'].str.isalpha()]

In [24]:
preprocessing_df.shape

(91288, 2)

## 3. 단어필터링

 - **나무위키**에서 찾은 욕설사전을 통해 단어필터링을 한다
 - **나무위키**는 여러사람이 수정할 수 있어 객관적인 기준이라고는 할 수 없지만, 여러사람들의 의견을 수용할 수 있다는 장점이있다.
 - 그러므로 **나무위키**는 여러사람의 보편적인 의견을 수용하여 나타낼 수 있는 중요한 지표라 생각하여 **나무위키**의 욕설사전을 사용한다.
 - 일일히 라벨링을 시도하기 전에 조금이라도 수작업을 줄이기위해 욕설이 포함된 악플에 라벨을 부여하여 수작업의 양을 줄인다.

In [25]:
preprocessing_df = preprocessing_df.reset_index()
preprocessing_df = preprocessing_df.drop(['index'], axis=1, inplace=False)

In [26]:
preprocessing_df['악플여부'] = np.NaN

In [27]:
filter_word_df = pd.read_excel('./02_필터링단어/필터링단어(나무위키).xlsx')

In [28]:
filter_word_df.shape[1]

13

In [29]:
for col in tqdm_notebook(range(filter_word_df.shape[1])):
    word_list = list(filter_word_df.iloc[:, col])[0].split(', ')
    label = filter_word_df.columns[col][-2]
    
    for filter_word in tqdm_notebook(word_list):
        preprocessing_df.loc[ preprocessing_df['댓글'].str.match('.*' + filter_word + '.*'), '악플여부' ] = label

In [30]:
preprocessing_df = preprocessing_df.reindex(columns = ['댓글', '악플여부', '출처'])

In [31]:
exist_label_df = preprocessing_df[preprocessing_df['악플여부'].notnull()]
not_exist_label_df = preprocessing_df[preprocessing_df['악플여부'].isnull()]

In [32]:
print(exist_label_df.shape)
print(not_exist_label_df.shape)

(8583, 3)
(82705, 3)


In [33]:
exist_label_df

,댓글,악플여부,출처
75,지랄을 한다 역격리 고맙다,2,./data\JTBCNEWS_인스타_댓글_1116.csv
86,지네때문에 온나라가 지랄낫는데,2,./data\JTBCNEWS_인스타_댓글_1116.csv
95,짱꺄나 쪽바리나,2,./data\JTBCNEWS_인스타_댓글_1116.csv
122,지랄한다미세먼지나 잡아라 중국에 말도 못하면서 말은 잘도 씨부린다,2,./data\JTBCNEWS_인스타_댓글_1116.csv
178,아무리 내새끼라도 다리병신 만들어버렸을듯,2,./data\JTBCNEWS_인스타_댓글_1116.csv
...,...,...,...
89851,진짜 ㅋㅋㅋ 댓글 지우는거 보소참 성격 병신 맞가,2,./data\함소원.csv
90081,진짜 대박이다언니힘내세요그냥이럴때 공인이고 모고 다 뒤로하시고 사이다 날려요저런애들...,2,./data\함소원.csv
90946,올바른길이란게 정해지지않는듯오빠가 열심히하시고 즐겁게 시간보내는것이 오빠만의 길인듯...,2,./data\홍석천.csv
91211,석천오빠님사랑해요 레스토랑 한번 가본 촌년이지만행복한 기억이였고늘 오빠가 잘 되길 ...,2,./data\홍석천.csv


## 4. 악플라벨링을 위해 6등분

In [37]:
# 수작업으로 라벨링을 하기 위해
# 팀원모두 동등한 데이터의 개수를 부여해야한다.

division = int(not_exist_label_df.shape[0]/6)

for i in range(6):
    if i == 5 :
        temp_data = not_exist_label_df.iloc[division*i : ]
    else : 
        temp_data = not_exist_label_df.iloc[division*i : division*(i+1)]
    print(str(i+1) + '번째 :', temp_data.shape[0])
        
    temp_data['댓글길이'] = temp_data['댓글'].apply(lambda x : len(str(x)))
    temp_data.sort_values(by='댓글길이', ascending=True, inplace=True)
    temp_data.drop(['댓글길이'], axis=1, inplace=True)
    temp_data.to_csv('./merge_data/division_data' + str(i+1) + '(2020-11-17).csv', encoding='utf-8-sig')

1번째 : 13784
2번째 : 13784
3번째 : 13784
4번째 : 13784
5번째 : 13784
6번째 : 13785


## 5. 단어 필터로 라벨링이 완료된 댓글 따로저장

In [34]:
exist_label_df.to_csv('./merge_data/exist_label_df_나무위키.csv', encoding='utf-8-sig')